In [490]:
import os
from pandas import Series
import pandasql
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARIMAResults
from statsmodels.tsa.statespace.sarimax import SARIMAX
from math import sqrt
from pandas import DataFrame
from scipy.stats import boxcox
import pandas as pd  # data frame operations
import numpy as np  # arrays and math functions
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy.stats import uniform  # for training-and-test split
import statsmodels.api as sm  # statistical models (including regression)
import statsmodels.formula.api as smf  # R-like model specification
import matplotlib.pyplot as plt  # 2D plotting
import seaborn as sns  # PROVIDES TRELLIS AND SMALL MULTIPLE PLOTTING
sns.set(color_codes=True)
from pandas import read_csv
import matplotlib as mpl
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import matplotlib.pyplot as plt
#import plotly.graph_objs as go
#from plotly.offline import init_notebook_mode, iplot
#init_notebook_mode(connected=True) 

In [491]:
# This funtion takes the file and checks & processes
def LoadFile(dataFileName):
    isExisted = os.path.isfile(dataFileName)
    if isExisted == True: # Read in the data when the file does exist
        df = pd.read_csv(dataFileName, encoding = 'latin-1')
    else: # Print our message when the file provide does not exist
        print("File name:", dataFileName, "does not exist in", os.getcwd(), "folder")
    return pd.DataFrame(df)

In [492]:
# Load files by using LoadFile function defines above
states_by_county_death = LoadFile("data/COVID-19_Death_Counts_in_the_United_States_by_County.csv")
state = LoadFile("data/State.csv")
us_counties_cases_deaths = LoadFile("data/us_counties_cases_deaths.csv")
detail_2018_county_health_rankings = LoadFile("data/detail_2018_county_health_rankings.csv")
overall_2018_county_health_ranking = LoadFile("data/overall_2018_county_health_ranking.csv")

In [493]:
states_by_county_death.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Date as of                 461 non-null    object
 1   First week                 461 non-null    object
 2   Last week                  461 non-null    object
 3   State                      461 non-null    object
 4   County name                461 non-null    object
 5   FIPS County Code           461 non-null    int64 
 6   Deaths involving COVID-19  461 non-null    int64 
 7   Deaths from All Causes     461 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 28.9+ KB


In [494]:
for column in states_by_county_death.columns:
    states_by_county_death.rename(columns={column: column.replace(' ', '_')}, inplace=True)

In [495]:
states_by_county_death['FIPS_County_Code'] = states_by_county_death['FIPS_County_Code'].astype(str)
states_by_county_death['FIPS_County_Code'] = states_by_county_death['FIPS_County_Code'].str.pad(width=5, side='left', fillchar='0')

In [496]:
states_by_county_death.head()

,Date_as_of,First_week,Last_week,State,County_name,FIPS_County_Code,Deaths_involving_COVID-19,Deaths_from_All_Causes
0,5/20/2020,2/1/2020,5/16/2020,AL,Etowah County,01055,11,432
1,5/20/2020,2/1/2020,5/16/2020,AL,Houston County,01069,24,542
2,5/20/2020,2/1/2020,5/16/2020,AL,Jefferson County,01073,79,2675
3,5/20/2020,2/1/2020,5/16/2020,AL,Lee County,01081,43,362
4,5/20/2020,2/1/2020,5/16/2020,AL,Madison County,01089,10,1149


In [497]:
states_by_county_death_agg = pandasql.sqldf("SELECT sc.State State, REPLACE(sc.County_name, ' County','' ) [Count_Name], sc.FIPS_County_Code [FIPS_Code], SUM(sc.[Deaths_involving_COVID-19]) [COVID_Death], SUM(sc.Deaths_from_All_Causes) [All_Cause_Death] FROM states_by_county_death sc GROUP BY sc.State, sc.County_name, sc.FIPS_County_Code")

In [498]:
states_by_county_death_agg = pandasql.sqldf("SELECT sc.*, s.StateName State_Name FROM states_by_county_death_agg sc LEFT JOIN state s ON rtrim(sc.State) = rtrim(ltrim(s.State))")

In [499]:
states_by_county_death_agg.head()

,State,Count_Name,FIPS_Code,COVID_Death,All_Cause_Death,State_Name
0,AL,Etowah,01055,11,432,Alabama
1,AL,Houston,01069,24,542,Alabama
2,AL,Jefferson,01073,79,2675,Alabama
3,AL,Lee,01081,43,362,Alabama
4,AL,Madison,01089,10,1149,Alabama


In [500]:
us_counties_cases_deaths.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [501]:
us_counties_cases_deaths.rename(columns={'fips': 'FIPS_Code'}, inplace=True)
us_counties_cases_deaths.rename(columns={'state': 'State_Name'}, inplace=True)
us_counties_cases_deaths.rename(columns={'county': 'County_Name'}, inplace=True)

In [524]:
us_counties_cases_deaths['FIPS_Code'] = us_counties_cases_deaths['FIPS_Code'].astype(str)
us_counties_cases_deaths['FIPS_Code'] = us_counties_cases_deaths['FIPS_Code'].str.replace('.0', '')
us_counties_cases_deaths['FIPS_Code'] =  us_counties_cases_deaths['FIPS_Code'].str.pad(width=5, side='left', fillchar='0')

In [522]:
us_counties_cases_deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167839 entries, 0 to 167838
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   date         167839 non-null  object
 1   County_Name  167839 non-null  object
 2   State_Name   167839 non-null  object
 3   FIPS_Code    167839 non-null  object
 4   cases        167839 non-null  int64 
 5   deaths       167839 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 7.7+ MB


In [525]:
us_counties_cases_deaths.head()

,date,County_Name,State_Name,FIPS_Code,cases,deaths
0,2020-01-21,Snohomish,Washington,00561,1,0
1,2020-01-22,Snohomish,Washington,00561,1,0
2,2020-01-23,Snohomish,Washington,00561,1,0
3,2020-01-24,Cook,Illinois,00131,1,0
4,2020-01-24,Snohomish,Washington,00561,1,0


In [505]:
overall_2018_county_health_ranking.rename(columns={'ï»¿FIPS': 'FIPS_Code'}, inplace=True)
overall_2018_county_health_ranking.rename(columns={'ï»¿FIPS': 'FIPS_Code'}, inplace=True)

In [506]:
us_counties_cases_deaths_agg = pandasql.sqldf("SELECT State_Name, County_Name, FIPS_Code, SUM(cases) COVID_Confirm FROM us_counties_cases_deaths GROUP BY State_Name, County_Name, FIPS_Code")

In [507]:
us_counties_cases_deaths_agg.head()

,State_Name,County_Name,FIPS_Code,COVID_Confirm
0,Alabama,Autauga,1001.0,2717
1,Alabama,Baldwin,1003.0,7949
2,Alabama,Barbour,1005.0,1981
3,Alabama,Bibb,1007.0,1689
4,Alabama,Blount,1009.0,1590


In [508]:
overall_2018_county_health_ranking['FIPS_Code'] = overall_2018_county_health_ranking['FIPS_Code'].astype(str)
overall_2018_county_health_ranking['FIPS_Code'] = overall_2018_county_health_ranking['FIPS_Code'].str.pad(width=5, side='left', fillchar='0')

In [509]:
overall_2018_county_health_ranking.head()

,FIPS_Code,State,County,Length of Life Rank,Quality of Life Rank,Health Behaviors Rank,Clinical Care Rank,Social & Economic Factors Rank,Physical Environment Rank,Health Outcomes Rank,Health Factors Rank
0,01001,Alabama,Autauga,18,5,12,15,3,51,11,8
1,01003,Alabama,Baldwin,4,4,3,7,8,14,3,3
2,01005,Alabama,Barbour,14,49,57,20,58,16,34,56
3,01007,Alabama,Bibb,53,25,39,42,42,29,41,37
4,01009,Alabama,Blount,17,12,11,41,13,54,14,19


In [510]:
detail_2018_county_health_rankings.rename(columns={'FIPS': 'FIPS_Code'}, inplace=True)
detail_2018_county_health_rankings.rename(columns={'State': 'StateName'}, inplace=True)

In [511]:
detail_2018_county_health_rankings['FIPS_Code'] = detail_2018_county_health_rankings['FIPS_Code'].astype(str)
detail_2018_county_health_rankings['FIPS_Code'] = detail_2018_county_health_rankings['FIPS_Code'].str.pad(width=5, side='left', fillchar='0')

In [512]:
detail_2018_county_health_rankings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 56 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   FIPS_Code                           3142 non-null   object 
 1   StateName                           3142 non-null   object 
 2   County                              3142 non-null   object 
 3   Number_Adult_Deaths                 3077 non-null   float64
 4   Age_Adjusted_Mortality              3077 non-null   float64
 5   Age_Adjusted_Mortality_Black        1334 non-null   float64
 6   Age_Adjusted_Mortality_Hispanic     797 non-null    float64
 7   Age_Adjusted_Mortality_White        1556 non-null   float64
 8   Number_Child_Deaths                 1931 non-null   float64
 9   Child_Mortality_Rate                1931 non-null   float64
 10  Child_Mortality_Rate_Black          498 non-null    float64
 11  Child_Mortality_Rate_Hispanic       359 non

In [513]:
detail_2018_county_health_rankings.head()

,FIPS_Code,StateName,County,Number_Adult_Deaths,Age_Adjusted_Mortality,Age_Adjusted_Mortality_Black,Age_Adjusted_Mortality_Hispanic,Age_Adjusted_Mortality_White,Number_Child_Deaths,Child_Mortality_Rate,...,Percent_African_American,Percent_American_Alaskan_Indian,Percent_Asian,Percent_Hawaiian_PacificIslander,Percent_Hispanic,Percent_White,Percent_ Not_Proficient_English,Percent_Female,Percent_Rural,Unnamed: 55
0,01001,Alabama,Autauga,827.0,450.0,532.0,NaN,445.0,31.0,56.2,...,19.0,0.5,1.1,0.1,2.7,75.0,0.8,51.3,42.0,NaN
1,01003,Alabama,Baldwin,2753.0,351.0,518.0,111.0,349.0,85.0,47.7,...,9.0,0.8,1.1,0.1,4.4,83.2,0.8,51.5,42.3,NaN
2,01005,Alabama,Barbour,418.0,434.0,477.0,NaN,395.0,15.0,66.6,...,47.5,0.6,0.5,0.2,4.2,46.5,1.5,46.9,67.8,NaN
3,01007,Alabama,Bibb,408.0,529.0,630.0,NaN,527.0,22.0,116.3,...,21.6,0.4,0.2,0.1,2.5,74.4,0.4,46.4,68.4,NaN
4,01009,Alabama,Blount,996.0,480.0,NaN,NaN,NaN,30.0,55.4,...,1.4,0.6,0.3,0.1,9.1,87.5,1.9,50.6,90.0,NaN


In [514]:
for column in overall_2018_county_health_ranking.columns:
    overall_2018_county_health_ranking.rename(columns={column: column.replace(' ', '_').replace('&_', '')}, inplace=True)
    if column == 'State':
        overall_2018_county_health_ranking.rename(columns={'State': 'StateName'}, inplace=True)

In [515]:
overall_2018_county_health_ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   FIPS_Code                     3142 non-null   object
 1   StateName                     3142 non-null   object
 2   County                        3142 non-null   object
 3   Length_of_Life_Rank           3142 non-null   object
 4   Quality_of_Life_Rank          3142 non-null   object
 5   Health_Behaviors_Rank         3142 non-null   object
 6   Clinical_Care_Rank            3142 non-null   object
 7   Social_Economic_Factors_Rank  3142 non-null   object
 8   Physical_Environment_Rank     3142 non-null   object
 9   Health_Outcomes_Rank          3142 non-null   object
 10  Health_Factors_Rank           3142 non-null   object
dtypes: object(11)
memory usage: 270.1+ KB


In [516]:
overall_2018_county_health_ranking.rename(columns={'State': 'StateName'}, inplace=True)

In [517]:
overall_2018_county_health_ranking.head()

,FIPS_Code,StateName,County,Length_of_Life_Rank,Quality_of_Life_Rank,Health_Behaviors_Rank,Clinical_Care_Rank,Social_Economic_Factors_Rank,Physical_Environment_Rank,Health_Outcomes_Rank,Health_Factors_Rank
0,01001,Alabama,Autauga,18,5,12,15,3,51,11,8
1,01003,Alabama,Baldwin,4,4,3,7,8,14,3,3
2,01005,Alabama,Barbour,14,49,57,20,58,16,34,56
3,01007,Alabama,Bibb,53,25,39,42,42,29,41,37
4,01009,Alabama,Blount,17,12,11,41,13,54,14,19
